In [1645]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score

Improt the dataset

In [1646]:
#setup display options
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 15)

df = pd.read_csv("..\data\data_cleaned.csv",index_col=[0])


In [1647]:
df.head()

,id,type of property,subtype of property,locality,latitude,longitude,street,postalCode,price,type of sale,number of bedrooms,living area,livable area,fully equipped kitchen,furnished,open fire,terrace,terrace area,garden,garden area,total property area,total land area,number of facades,swimming pool,state of the building,Gemeente,Province,Population( per province),Area(per province),Density,kitchen,price per sqr
0,10689265,HOUSE,MIXED_USE_BUILDING,Koekelberg,50.863091,4.329928,Rue du Noyer 86,1081,575000.0,residential_sale,3,0,220.0,installed,False,False,True,0.0,False,0.0,220.0,71.0,2.0,False,good,Koekelberg,Brussel,1208542.0,162.4,7442.0,NaN,2613.636364
1,10687297,HOUSE,EXCEPTIONAL_PROPERTY,Beveren,51.224437,4.246054,NaN,9120,695000.0,residential_sale,4,28,269.0,hyper_equipped,False,False,True,41.0,True,300.0,269.0,583.0,3.0,False,as_new,Beveren,Oost-Vlaanderen,1515064.0,3007.0,504.0,Hyper equipped,2583.643123
2,10686031,HOUSE,HOUSE,Seraing,50.598224,5.518784,Place Merlot 6,4100,144000.0,residential_sale,2,14,105.0,semi_equipped,False,False,True,0.0,True,25.0,105.0,105.0,2.0,False,good,Seraing,Liège,1106992.0,3857.0,288.0,Semi equipped,1371.428571
3,10686030,HOUSE,HOUSE,Seraing,50.601548,5.510799,Place Merlot 6,4100,199000.0,residential_sale,4,25,125.0,installed,False,False,True,108.0,True,28.0,125.0,229.0,2.0,False,good,Seraing,Liège,1106992.0,3857.0,288.0,NaN,1592.000000
4,10686027,HOUSE,VILLA,Herzele,50.891145,3.857915,Hoogstraat 20,9550,539000.0,residential_sale,4,65,370.0,hyper_equipped,False,False,True,60.0,False,0.0,370.0,911.0,4.0,False,as_new,Herzele,Oost-Vlaanderen,1515064.0,3007.0,504.0,Hyper equipped,1456.756757


In [1648]:
df["subtype of property"].value_counts()
df["latitude"].isna().value_counts()

latitude
False    12976
True      2694
Name: count, dtype: int64

Data cleaning!
1. fillna
2. remove castles
3. scaling certain data
4. get dummies


In [1649]:
#replace categorical data with number
df.drop(df["kitchen"].isna().index)

#remove the properties without coordinates
lon_la = (df["latitude"].isna()) | (df["longitude"].isna())
df.drop(df.loc[lon_la].index,inplace=True)

#remove castles
castle = df[df["subtype of property"]=="CASTLE"].index
df.drop(castle,inplace=True)


def convert(n):
    return int(n/100)
df["digit"]=df["postalCode"].agg(convert)

#get dummies for catagorical data:
get_dummies = ["Province","type of property","kitchen","subtype of property","state of the building","garden","terrace","digit"]
df =pd.get_dummies(data=df, columns=get_dummies)

df.drop(columns=["fully equipped kitchen"],inplace=True)

#replace True with 1, and False with 0, turn them into numerical data
df.replace([True,False],[1,0],inplace=True)


In [1650]:
df.shape

(12970, 153)

Remove outliners and castles

In [1651]:
def remove_outliners(df,columns,n_std):
    for col in columns:

        mean = df[col].mean()
        sd = df[col].std()
        df = df[(df[col] <= mean+(n_std*sd))]
        df = df[(df[col] >= mean-(n_std*sd))]

    return df

# def remove_outliner(df, col_names):
#     for col_name in col_names:
#         q1 = df[col_name].quantile(0.25)
#         q3 = df[col_name].quantile(0.75)
#         iqr = q3-q1 #Interquartile range
#         fence_low  = q1-1.5*iqr
#         fence_high = q3+1.5*iqr
#         df = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]

#     return df



In [1652]:
# print(df["price"].std())
# print(df["price"].mean())
df = remove_outliners(df, ["number of bedrooms","livable area","living area",
                           "total property area","garden area","total land area","terrace area"],3)
df.shape

(12299, 153)

Data formatting: 
1.normalize the data
2.create X,y, split them into training and testing

In [1653]:
# # fix the distributions of number of bedrooms
# df["number of bedrooms"] = np.log(df["number of bedrooms"] + 1)

#normalize these columns
columns_to_normalize = ["latitude","longitude","living area","livable area",
                        "garden area","total property area","total land area","number of facades","number of bedrooms","terrace area"]

# print(df["price"].sort_values())

In [1654]:
# price would be the target
columns_to_drop = ["id","locality","street","price","type of sale"
                   ,"Density","Area(per province)",
                   "Population( per province)","price per sqr","open fire","postalCode","Gemeente"
                   ]
X=df.drop(columns=columns_to_drop,axis=1)
df=preprocessing.normalize(df[columns_to_normalize])
X = .to_numpy()
y = df.loc[:,"price"].to_numpy().reshape(-1,1)
display(df.drop(columns=columns_to_drop,axis=1).head())
#Now normalize the data
# X=preprocessing.normalize(X_raw) 
# print(X.shape,y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [ ]:
print(X.shape, X_train.shape, y_train.shape)


(12299, 141) (9839, 141) (9839, 1)


visualization of the raw_data

In [ ]:

# df[columns_to_select].hist(figsize=(15,8))



Model selection :Linear Regression

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

# y_pred = reg.predict(X_test)
print(reg.score(X_train,y_train))
print(reg.score(X_test,y_test))



0.4514914871452783
0.4090386603355106


Decision Tree?


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

clf = DecisionTreeClassifier(max_depth=50,max_leaf_nodes=50)
clf.fit(X_train,y_train)
clf.score(X_train,y_train)
clf.score(X_test,y_test)



0.026829268292682926

Try XGboost

In [ ]:
from xgboost import XGBClassifier
model= XGBClassifier()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 1146 1147 1148], got [  22000.   39900.   40000. ... 5650000. 5995000. 8100000.]